# pip install

In [ ]:
!pip install gdown
!pip install scikit-learn
!pip install joblib

# Load Raw Data

In [1]:
from utilities.data_processor import DataPreProcessor

dpp = DataPreProcessor()

# Load the data
dpp.load_data(download=True)

# show the row data info
dpp.show_data_info()

# preprocess the data
dpp.preprocess_data(fill_blank=True, fill_mode='mode')
 
# show the preprocessed data info
dpp.show_data_info()

# plt the remaining features
dpp.show_feature_plt()

/Users/zhangzhe/Documents/GitHub/STAT7008_Group6b_2024/utilities/data_processor.py:130: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data: pd.DataFrame = pd.read_csv(self.input_path, index_col=0)
/Users/zhangzhe/Documents/GitHub/STAT7008_Group6b_2024/utilities/data_processor.py:208: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  self.data[i].fillna(self.data[i].mode()[0], inplace=True)


Data Preprocessing Summary:
Rows Deleted: 1126
	511 rows deleted due to del application_type 'JOINT'
	615 rows deleted due to del rows with insignificant missing values
Columns Deleted: 37
	4 columns deleted due to del columns related to 'joint', namely: annual_inc_joint, dti_joint, verification_status_joint, application_type
	19 columns deleted due to del columns with more than 20% missing values, namely: desc, mths_since_last_delinq, mths_since_last_record, next_pymnt_d, mths_since_last_major_derog, open_acc_6m, open_il_6m, open_il_12m, open_il_24m, mths_since_rcnt_il, total_bal_il, il_util, open_rv_12m, open_rv_24m, max_bal_bc, all_util, inq_fi, total_cu_tl, inq_last_12m
	1 columns deleted due to del columns 'policy_code'
	13 columns deleted due to del columns after analyzing (analyzing procedures are in the file 'data_analysis.ipynb'), namely: member_id, url, issue_d, earliest_cr_line, grade, last_credit_pull_d, emp_title, title, zip_code, addr_state, last_pymnt_d, pymnt_plan, emp_

# DataLoader

In [2]:
print(dpp.get_data().shape)

(886253, 36)


In [3]:
from utilities import DataLoader, maxmin_scaler
from utilities.data_processor import CatBoostDataLoader

'''
使用catboost进行降维
dl = CatBoostDataLoader(dpp.get_data())
dl.run_catboost()
'''
dl = DataLoader(dpp.get_data())

dl.split_data(MAXMIN=True)

dl.data_filter(mode='pca')

x_train_filtered, x_test_filtered, y_train, y_test = dl.get_filtered_data()

Selected features (15): ['out_prncp_inv', 'out_prncp', 'total_rec_prncp', 'last_pymnt_amnt', 'tot_cur_bal', 'recoveries', 'int_rate', 'total_pymnt', 'total_rec_int', 'funded_amnt_inv', 'total_pymnt_inv', 'inq_last_6mths', 'sub_grade', 'total_rev_hi_lim', 'revol_bal']


In [4]:
print(x_train_filtered.shape, x_test_filtered.shape, y_train.shape, y_test.shape)
print(x_train_filtered.isna().sum().sum(), x_test_filtered.isna().sum().sum())

(62038, 15) (26588, 15) (62038,) (26588,)
0 0


In [5]:
x_train_filtered.head(3)

,out_prncp_inv,out_prncp,total_rec_prncp,last_pymnt_amnt,tot_cur_bal,recoveries,int_rate,total_pymnt,total_rec_int,funded_amnt_inv,total_pymnt_inv,inq_last_6mths,sub_grade,total_rev_hi_lim,revol_bal
id,,,,,,,,,,,,,,,
68202755,0.657143,0.657143,0.000000,0.000000,0.076925,0.0,0.109421,0.000000,0.000000,0.657143,0.000000,0.035714,0.882353,0.072347,0.011155
59602975,0.407421,0.407421,0.021150,0.009312,0.012037,0.0,0.294466,0.023815,0.027251,0.428571,0.023815,0.035714,0.705882,0.030056,0.009861
45314462,0.906567,0.906567,0.093433,0.025974,0.041738,0.0,0.640896,0.149902,0.235374,1.000000,0.149902,0.035714,0.323529,0.035287,0.015242
